In [ ]:
import os
import tensorflow as tf

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [ ]:
vgg = tf.keras.applications.VGG19(include_top=False, input_shape=(400, 400, 3), weights='imagenet')
vgg.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400, 400, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 400, 400, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 400, 400, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 200, 200, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 200, 200, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 200, 200, 128)     147584    
______________________________________________________________

In [ ]:
layer = vgg.get_layer('block5_conv1')
print(layer)

In [ ]:
output = vgg.get_layer('block5_conv1').output
print(output.shape)

(None, 25, 25, 512)


In [ ]:
output = vgg.get_layer('block5_conv2').output
print(output.shape)

(None, 25, 25, 512)


In [ ]:
outputs = [vgg.get_layer('block5_conv1').output, vgg.get_layer('block5_conv2').output]
print(outputs)

[<KerasTensor: shape=(None, 25, 25, 512) dtype=float32 (created by layer 'block5_conv1')>, <KerasTensor: shape=(None, 25, 25, 512) dtype=float32 (created by layer 'block5_conv2')>]


In [ ]:
content_layers = ['block5_conv1']

In [ ]:
def get_vgg_layers(layer_names):
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False

    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], [outputs])

    return model

In [ ]:
# create the model:
content_layers = ['block5_conv2']

content_extractor = get_vgg_layers(content_layers)

In [ ]:
data = tf.random.normal((1, 400, 400, 3))
content_outputs = content_extractor(data)
print(type(content_outputs))
print(content_outputs[0].shape)

<class 'list'>
(1, 25, 25, 512)


In [ ]:

# Look at the statistics of each layer's output
for name, output in zip(content_layers, content_outputs):
    print(name)
    print("  shape: ", output.numpy().shape)
    print("  min: ", output.numpy().min())
    print("  max: ", output.numpy().max())
    print("  mean: ", output.numpy().mean())
    print()